In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from d2l import torch as d2l
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.models as models

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
try:
    # if use GPU, use it
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {torch.cuda.get_device_name()} for training." if torch.cuda.is_available() else "Using CPU for training")
except:
    print(f"No GPU found. Using CPU for training.")
    device = torch.device("cpu")

Using NVIDIA GeForce RTX 3060 Ti for training.


In [3]:
dataset_path = 'Dataset/'
class_names = sorted(os.listdir(dataset_path))
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
batch_size = 32
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [4]:
class_names

['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

In [5]:
# Processing images and labels in each folder
for batch in data_loader:
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)

In [6]:
total_images, total_labels = 0, 0
for batch in data_loader:
    images, labels = batch
    total_images += images.size(0)
    total_labels += len(labels)

In [7]:
print(f"Total images: {total_images}", f"\nTotal labels: {total_labels}")    

Total images: 10287 
Total labels: 10287


In [8]:
# Train-Test Split
train_data, test_data, train_labels, test_labels = train_test_split(dataset, dataset.targets, test_size=0.15, random_state=42)
# Train-Validation Split
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.15, random_state=42)

# Creating new data_loader
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

# Checking shape of datasets
print("Train: ", len(train_data))
print("Validation: ", len(val_data))
print("Test: ", len(test_data))

Train:  7431
Validation:  1312
Test:  1544


In [9]:
# Resnet
class ResNet(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=pretrained)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

In [10]:
num_classes = 4
resnet_model = ResNet(num_classes=num_classes, pretrained=True)

In [11]:
# Identify train and validation function
def train(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

def validate(model, criterion, val_loader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        epoch_loss = running_loss / len(val_loader)
        epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

In [14]:
lr = 0.001
num_epochs = 40
optimizer = optim.Adam(resnet_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
resnet_model.to(device)

ResNet(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [15]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(resnet_model, criterion, optimizer, train_loader, device)
    val_loss, val_acc = validate(resnet_model, criterion, val_loader, device)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {100 * train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {100 * val_acc:.2f}%')

Epoch [1/40], Train Loss: 0.0100, Train Acc: 99.58%, Val Loss: 0.1492, Val Acc: 97.87%
Epoch [2/40], Train Loss: 0.0032, Train Acc: 99.89%, Val Loss: 0.1373, Val Acc: 98.17%
Epoch [3/40], Train Loss: 0.0013, Train Acc: 99.99%, Val Loss: 0.1394, Val Acc: 98.02%
Epoch [4/40], Train Loss: 0.0013, Train Acc: 99.96%, Val Loss: 0.1322, Val Acc: 98.25%
Epoch [5/40], Train Loss: 0.0012, Train Acc: 99.97%, Val Loss: 0.1312, Val Acc: 98.02%
Epoch [6/40], Train Loss: 0.0009, Train Acc: 99.99%, Val Loss: 0.1281, Val Acc: 98.17%
Epoch [7/40], Train Loss: 0.0005, Train Acc: 99.99%, Val Loss: 0.1343, Val Acc: 98.17%
Epoch [8/40], Train Loss: 0.0008, Train Acc: 99.99%, Val Loss: 0.1281, Val Acc: 98.17%
Epoch [9/40], Train Loss: 0.0008, Train Acc: 99.97%, Val Loss: 0.1239, Val Acc: 98.17%
Epoch [10/40], Train Loss: 0.0006, Train Acc: 99.97%, Val Loss: 0.1343, Val Acc: 98.25%


KeyboardInterrupt: 

In [21]:
# Predictions

# Selecting 20 random image from the test set
random_indices = random.sample(range(len(test_data)), 20)

for idx in random_indices:
    image, label = test_data[idx]

    # Export the image to the model
    image = image.unsqueeze(0).to(device)
    output = resnet_model(image)

    # Print by taking the model's prediction and the actual label
    _, predicted = torch.max(output, 1)
    predicted_label = class_names[predicted.item()]
    true_label = class_names[label]

    print(f"Label of Image: {true_label}, Prediction of Model: {predicted_label}")

Label of Image: meningioma_tumor, Prediction of Model: meningioma_tumor
Label of Image: glioma_tumor, Prediction of Model: glioma_tumor
Label of Image: no_tumor, Prediction of Model: no_tumor
Label of Image: no_tumor, Prediction of Model: no_tumor
Label of Image: meningioma_tumor, Prediction of Model: meningioma_tumor
Label of Image: meningioma_tumor, Prediction of Model: meningioma_tumor
Label of Image: meningioma_tumor, Prediction of Model: meningioma_tumor
Label of Image: no_tumor, Prediction of Model: no_tumor
Label of Image: glioma_tumor, Prediction of Model: glioma_tumor
Label of Image: glioma_tumor, Prediction of Model: glioma_tumor
Label of Image: no_tumor, Prediction of Model: no_tumor
Label of Image: glioma_tumor, Prediction of Model: glioma_tumor
Label of Image: meningioma_tumor, Prediction of Model: meningioma_tumor
Label of Image: glioma_tumor, Prediction of Model: meningioma_tumor
Label of Image: pituitary_tumor, Prediction of Model: pituitary_tumor
Label of Image: pituit

In [22]:
# Save model
scripted_model = torch.jit.trace(resnet_model, torch.randn(1, 3, 224, 224).to(device)) # convert torch script
scripted_model.save('resnet_model.pt')